In [17]:
import pickle
import gzip
import pandas as pd
import numpy as np

In [2]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/del_X_train.pickle','rb') as f:
    del_X_train = pickle.load(f)

In [3]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/bigtrain_y.pickle','rb') as f:
    del_y_train = pickle.load(f)

In [4]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/pad_X_test.pickle','rb') as f:
    pad_X_test = pickle.load(f)

In [5]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/pad_X_train.pickle','rb') as f:
    pad_X_train = pickle.load(f)

In [6]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/token_X_test.pickle','rb') as f:
    token_X_test = pickle.load(f)

In [7]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/pad_X_val.pickle','rb') as f:
    pad_X_val = pickle.load(f)

In [8]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/preprocessing_data/bigval_y.pickle','rb') as f:
    bigval_y = pickle.load(f)

In [9]:
data = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/data/Pre3.csv')

**모델3**

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau
from tensorflow.keras.regularizers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam, RMSprop
import tensorflow as tf
import os
tf.random.set_seed(777)

In [11]:
path = "/content/drive/MyDrive/Colab Notebooks"
vocab_size= 18658
max_len = 300
val_rate = 0.2

In [12]:
model3 = Sequential()
model3.add(Embedding(vocab_size, 64, input_length = max_len))
model3.add(Conv1D(32, 5, activation='relu'))
model3.add(Conv1D(32, 5, activation='relu'))
model3.add(Conv1D(32, 5, activation='relu'))
model3.add(MaxPooling1D(pool_size=4))
model3.add(LSTM(16))
model3.add(Dropout(0.4))
model3.add(Dense(21, activation='softmax'))
model3.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 64)           1194112   
_________________________________________________________________
conv1d (Conv1D)              (None, 296, 32)           10272     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 292, 32)           5152      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 288, 32)           5152      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 72, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 16)                3136      
_________________________________________________________________
dropout (Dropout)            (None, 16)                0

In [ ]:
model3.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr = .0005), metrics=['acc'])

In [ ]:
callback_ear = [EarlyStopping(monitor = 'val_acc', mode = 'max', patience = 2, verbose = 1), ModelCheckpoint(filepath =  os.path.join(path, '1028_3_big1.h5'), monitor = 'val_acc', save_best_only = True, mode = 'max', verbose = 1)]

history3 = model3.fit(pad_X_train, del_y_train, epochs=4, batch_size=64, validation_split=val_rate, verbose = 1,shuffle = True, callbacks= callback_ear)

Epoch 1/4
14245/14245 [==============================] - 228s 14ms/step - loss: 1.4258 - acc: 0.6053 - val_loss: 0.7264 - val_acc: 0.8034

Epoch 00001: val_acc improved from -inf to 0.80343, saving model to /content/drive/MyDrive/Colab Notebooks/1028_3_big1.h5
Epoch 2/4
14245/14245 [==============================] - 196s 14ms/step - loss: 0.8354 - acc: 0.7871 - val_loss: 0.6730 - val_acc: 0.8175

Epoch 00002: val_acc improved from 0.80343 to 0.81751, saving model to /content/drive/MyDrive/Colab Notebooks/1028_3_big1.h5
Epoch 3/4
14245/14245 [==============================] - 192s 13ms/step - loss: 0.7795 - acc: 0.7994 - val_loss: 0.6566 - val_acc: 0.8177

Epoch 00003: val_acc improved from 0.81751 to 0.81765, saving model to /content/drive/MyDrive/Colab Notebooks/1028_3_big1.h5
Epoch 4/4
14245/14245 [==============================] - 192s 13ms/step - loss: 0.7606 - acc: 0.8042 - val_loss: 0.6512 - val_acc: 0.8188

Epoch 00004: val_acc improved from 0.81765 to 0.81884, saving model to /

In [12]:
model3 = load_model( os.path.join(path, '1028_3_big1.h5')) #0.70633,

In [14]:
pred_val = model3.predict(pad_X_val)

In [15]:
import numpy as np


score_board = []
for true,pred in zip(bigval_y,pred_val):
  if np.argmax(true) == np.argmax(pred):
    score_board.append(1)
  else:
    score_board.append(0)
print('Accuracy : ',np.array(score_board).sum()/len(score_board) * 100)

Accuracy :  81.87171622446179


In [19]:
big = sorted(data['Big'].unique())

In [21]:
val = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_val.csv')

In [24]:
val

,big2,big3
0,G,G
1,C,C
2,F,M
3,A,A
4,D,D
...,...,...
488381,C,C
488382,L,L
488383,M,M
488384,C,C


In [23]:
pred_big_val = []
for i in pred_val:
  pred_idx = np.argmax(i)
  pred_big_val.append(big[pred_idx])

val['big3'] = pred_big_val


In [25]:
val.to_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_val.csv',index=False)

In [13]:
test = pd.read_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_test.csv')

In [14]:
test.shape

(19159, 1)

In [15]:
pred_X_test = model3.predict(pad_X_test)

In [20]:
pred_big_test = []
for i in pred_X_test:
  pred_idx = np.argmax(i)
  pred_big_test.append(big[pred_idx])


In [23]:
test['big3']=pred_big_test
test

,big2,big3
0,F,F
1,C,C
2,F,F
3,L,L
4,A,A
...,...,...
19154,M,M
19155,C,C
19156,A,A
19157,J,J


In [24]:
test.to_csv('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_test.csv',index=False)

In [25]:
pred_train = model3.predict(pad_X_train)

In [26]:
pred_big_train = []
for i in pred_train:
  pred_idx = np.argmax(i)
  pred_big_train.append(big[pred_idx])



In [27]:
with gzip.open('/content/drive/MyDrive/애쓰는 감자/코드/ysb/output/big_predict_train3.pickle', 'wb') as f:
    pickle.dump(pred_big_train, f)

In [ ]:
#with gzip.open('/content/drive/MyDrive/애쓰는 감자/data/ysb_predict_data/m3_pred.pickle', 'wb') as f:
 #   pickle.dump(pred_val, f)